# IFT 6758 - Devoir 4

### Interprétabilité du modèle, Explicabilité, Sélection et Ingénierie de caractéristiques

**Ce devoir a pour objectif de vous donner plus d'expérience pratique dans les domaines de l'interprétabilité du modèle, de la sélection des caractéristiques et de l'ingénierie des caractéristiques.**

Évaluation dans ce jupyter notebook :

- Sorties et graphiques
- Résultats imprimés issus de l'exécution de fonctions
- Questions à réponse courte

In [ ]:
import pandas as pd
import numpy as np
import eli5
from sklearn.ensemble import RandomForestRegressor
from hw4 import *

# Partie 1 : Interprétabilité et Explicabilité du Modèle

## 1. Chargement des Données

### Chargement de l'ensemble de données des réadmissions

Nous commencerons par charger notre ensemble de données et vérifier un petit échantillon.

In [ ]:
path = "data/hospital.csv"
raw_df = load_data(path)

In [ ]:
# Vérification d'un échantillon de 5 patients
np.random.seed(0)
raw_df.sample(5)

Nous pouvons vérifier les noms de toutes les colonnes de notre ensemble de données :

In [ ]:
# Vérification des noms des 65 colonnes de notre ensemble de données
raw_df.columns

Nous pouvons également vérifier leurs types de données :

In [ ]:
# Vérification des types de données du dataframe brut
raw_df.dtypes

Ici, vous avez un bref aperçu de la manière d'interpréter chaque nom de colonne :

- La colonne portant le nom `is_readmitted` est notre cible de prédiction. C'est ce que notre modèle devrait être en mesure de classifier correctement. Remarquez que les valeurs de cette colonne sont booléennes et que nous devrons les mapper en 1 et 0 avant d'entraîner notre modèle.
- Les colonnes avec le mot `diag` indiquent le code de diagnostic de la maladie ou des maladies pour lesquelles le patient a été admis. Par exemple, `diag_1_428` signifie que le médecin a donné le diagnostic de la première maladie comme "428". Ces codes pourraient être consultés dans un répertoire de codes médicaux, mais sans plus d'informations médicales, ils ne signifieraient rien pour nous de toute façon.
- Les colonnes avec des noms comme `glimepiride_No` signifient que le patient n'a pas pris le médicament `x`. Dans cet exemple, le médicament est `glimepiride`. Si cette caractéristique avait une valeur de False, cela signifierait que le patient a pris le médicament `glimepiride`.
- Les caractéristiques dont les noms commencent par `medical_specialty` décrivent la spécialité du médecin qui voit le patient. Les valeurs de ces champs sont toutes `True` ou `False`.

Nous vérifions s'il y a des valeurs manquantes dans notre ensemble de données.

In [ ]:
# Vérifier s'il y a des valeurs manquantes (NA)
raw_df.isna().any(axis=1).sum() 

Enfin, nous encoderons notre cible de prédiction. Complétez la méthode `encode_target_column()` dans `hw4.py`.

In [ ]:
# TODO : compléter la méthode encode_target_column() dans hw4.py
df = encode_target_column(raw_df)
df.head(5)

## 2. Interprétation du Modèle

Imaginons que nous ayons choisi un modèle simple qui prédit si un patient sera réadmis à l'hôpital, mais les médecins (nos principales parties prenantes) disent qu'ils ne savent pas comment évaluer un modèle et ils aimeraient que nous leur montrions des preuves que le modèle est en ligne avec leur intuition médicale.

Ils sont très occupés à sauver des vies et ils n'ont pas le temps pour des graphiques ou des rapports compliqués. Nous devons produire une représentation qui résume notre aperçu du modèle en 1 ou 2 graphiques.

Tout d'abord, nous devons préparer notre ensemble de données pour l'entraînement du modèle et l'évaluation du modèle. Nous utiliserons la fonction `split_data()` de notre fichier `hw4.py` pour diviser notre ensemble de données en X_train, y_train, X_val et y_val. Nous sommes de jeunes scientifiques des données et nous croyons que toutes les fonctionnalités sont utiles pour prédire si un patient sera réadmis (je sais que vous savez que c'est faux, mais jouons le jeu pour l'exercice).

In [ ]:
# Diviser l'ensemble de données en ensembles d'entraînement et de validation
X_train, X_val, y_train, y_val = split_data(df, "is_readmitted")

Le modèle que nous avons choisi est un modèle de `Random Forest`. Nous allons entraîner et tester notre modèle sur les subdivisions de la cellule précédente. Ici, vous devrez compléter la méthode `train_random_forest()` du fichier `hw4.py`.

In [ ]:
# TODO: compléter la méthode train_random_forest() dans hw4.py
firstModel = train_random_forest(X_train, y_train)

Nous sommes novices dans le domaine de la science des données médicales, mais nous savons qu'un rapport de classification pourrait nous aider à montrer que le modèle effectue son travail. Ici, vous devrez compléter la méthode `evaluate_model()` de `hw4.py`, car elle nous aidera à obtenir la précision de notre modèle et notre rapport de classification.

In [ ]:
# TODO: complete the method evaluate_model() in hw4.py
acc, report = evaluate_model(firstModel, X_val,y_val)
print(f"The accuracy of our model is: {acc}")
print(report)

### 2.1 Importance des Caractéristiques

Ouch ! Nous n'avons pas fait très bien. Cependant, la science des données est un processus d'amélioration continue, et nous savons que nous pourrions améliorer ce modèle si nous comprenons comment nos caractéristiques interagissent avec notre modèle.

Nous restons calmes et nous nous souvenons que nous pouvons interroger notre modèle pour identifier les caractéristiques qui ont le plus d'impact sur la prédiction. Il existe plusieurs façons de le faire, mais nous décidons de travailler avec l'`importance par permutation` (elle est rapide à calculer, largement utilisée et comprise, et elle est conforme aux propriétés que nous souhaiterions qu'une mesure d'importance des caractéristiques ait).

L'importance par permutation est calculée après qu'un modèle a été ajusté. Nous ne changerons pas le modèle ni les prédictions que nous obtiendrions pour un ensemble de caractéristiques donné. Cette mesure nous aidera à répondre à la question : Si je mélange aléatoirement une colonne unique des données de validation, en laissant la cible et toutes les autres colonnes en place, comment cela affecterait-il la précision des prédictions dans ces données maintenant mélangées ? L'intuition derrière cela est que la précision du modèle souffre particulièrement si nous mélangeons une colonne sur laquelle le modèle s'appuie fortement pour ses prédictions.

Ici, vous devrez compléter la méthode `calculate_permutation_importance()`, qui nous permettra de calculer les importances par permutation des caractéristiques utilisées. Nous les montrerons ensuite à l'aide de la fonction `eli5.show_weights()`.

In [ ]:
# TODO: compléter la méthode calculate_permutation_importance dans hw4.py
feature_names = X_val.columns.tolist()
perm = calculate_permutation_importance(firstModel, X_val, y_val)

In [ ]:
eli5.show_weights(perm, feature_names = X_val.columns.tolist())

**Question 1**

1-. Quelle semble être la caractéristique la plus importante ?

*Votre réponse*

---------------------------------------------------------------

**Interprétation des Importances par Permutation**

Les valeurs en haut sont les caractéristiques les plus importantes, et celles en bas sont les moins importantes.

Le premier nombre dans chaque ligne indique de combien les performances du modèle ont diminué avec un mélange aléatoire (dans ce cas, en utilisant "l'exactitude" comme mesure de performance). Le nombre après le ± mesure comment les performances ont varié d'un mélange à l'autre.

Il est possible de voir des valeurs négatives pour les importances par permutation. Dans ces cas, les prédictions sur les données mélangées (ou bruitées) se sont avérées plus précises que sur les données réelles. Cela se produit lorsque la caractéristique n'avait pas d'importance (elle aurait dû avoir une importance proche de 0), mais le hasard a fait que les prédictions sur les données mélangées étaient plus précises.

----------------------------------------------------

### 2.2 Graphiques de Dépendance Partielle

Nous montrons nos conclusions aux médecins. Ils ne semblent pas trop impressionnés par les performances de notre modèle, mais ils aimeraient en savoir plus sur la caractéristique que le modèle a identifiée comme la plus importante. Nous allons créer un graphique pour leur montrer comment la caractéristique principale affecte les prédictions du modèle. Pour ce faire, vous devrez compléter la méthode `plot_partial_dependence()` de `solution.py`, qui nous aidera à afficher un graphique de dépendance partielle.

Alors que l'importance des caractéristiques montre quelles variables affectent le plus les prédictions, les graphiques de dépendance partielle montrent comment une caractéristique affecte les prédictions. Si vous êtes familier avec les modèles de régression linéaire ou logistique, les graphiques de dépendance partielle peuvent être interprétés de manière similaire aux coefficients de ces modèles. Cependant, les graphiques de dépendance partielle sur des modèles sophistiqués peuvent capturer des motifs plus complexes que les coefficients des modèles simples.

Les graphiques de dépendance partielle sont calculés après qu'un modèle a été ajusté. Le modèle est ajusté sur des données réelles qui n'ont pas été manipulées de quelque manière que ce soit.

In [ ]:
# TODO: compléter la méthode plot_partial_dependence() dans hw4.py et attribuer le nom de la caractéristique la plus importante que nous avons trouvée à la variable topFeature.
topFeature="number_inpatient"
plot_partial_dependence(firstModel,X_val,topFeature)

Nous montrons notre graphique aux médecins et ils pensent que c'est un bon signe que l'augmentation du nombre de procédures hospitalières entraîne une augmentation des prédictions. Mais ils ne peuvent pas dire à partir de ce graphique si ce changement est important ou non. Ils aimeraient que nous en fassions un semblable pour `time_in_hospital` pour voir comment cela se compare. Voyons comment ça se passe :

In [ ]:
plot_partial_dependence(firstModel,X_val,"time_in_hospital")

Il semble que `time_in_hospital` n'ait aucune importance du tout. La différence entre la valeur la plus basse sur le graphique de dépendance partielle et la valeur la plus élevée est d'environ 5 %.

Si c'est ce que notre modèle a conclu, les médecins le croiront. Mais pour eux, cela semble si faible. Est-il possible que les données soient incorrectes, ou notre modèle fait-il quelque chose de plus complexe que ce à quoi ils s'attendent ?

Ils aimeraient que nous leur montrions le taux de réadmission brut pour chaque valeur de `time_in_hospital` pour voir comment cela se compare au graphique de dépendance partielle.

In [ ]:
# TODO: compléter la méthode plot_mean_readmission_vs_time() dans hw4.py.
plot_mean_readmission_vs_time(X_train,y_train)

## 3. Explicabilité du Modèle (Bonus)

### 3.1 Valeurs SHAP

Après avoir visionné cela, les médecins sont convaincus que nous avons les bonnes données, et un aperçu du modèle semblait raisonnable pour un premier essai. Il est temps de transformer cela en un produit fini qu'ils peuvent utiliser et tester (dans la vraie vie, cela ne se produirait pas, car les modèles doivent passer une série de contrôles et de réglementations stricts avant d'être utilisés dans des environnements médicaux réels, mais l'idée ici est de vous montrer comment utiliser ces outils et d'être suffisamment familiarisé pour les appliquer dans notre projet de cours). Plus précisément, l'hôpital souhaite que vous créiez une fonction `main_factors()` qui fait ce qui suit :
- Prend une seule ligne de données de patient (du même format que nos données traitées)
- Crée une visualisation montrant quelles caractéristiques de ce patient ont augmenté leur risque de réadmission, quelles caractéristiques l'ont diminué, et dans quelle mesure ces caractéristiques ont de l'importance.

Il n'est pas important de montrer chaque caractéristique avec chaque impact minuscule sur le risque de réadmission. Il est acceptable de se concentrer uniquement sur les caractéristiques les plus importantes pour ce patient.

--------------------------------------

Jusqu'à présent, nous avons travaillé avec certaines techniques pour extraire des informations générales à partir d'un modèle d'apprentissage automatique. Mais que se passe-t-il si nous voulons décomposer le fonctionnement du modèle pour une prédiction individuelle ?

Les valeurs SHAP (un acronyme de SHapley Additive exPlanations) décomposent une prédiction pour montrer l'impact de chaque caractéristique. Où pourriez-vous utiliser cela ? Imaginez les scénarios suivants :

    - Un modèle indique qu'une banque ne devrait pas accorder un prêt à quelqu'un, et la banque est légalement tenue d'expliquer les raisons de chaque refus de prêt (nous commençons à voir de plus en plus de réglementations exigeant de l'explicabilité des modèles pour ce type d'applications en ML).
    - Un fournisseur de soins de santé souhaite identifier les facteurs qui influent sur le risque de contracter une maladie pour chaque patient, afin de pouvoir intervenir directement sur ces facteurs de risque avec des interventions médicales ciblées (c'est également important d'un point de vue réglementaire, car les fournisseurs de soins de santé doivent fournir des preuves claires sur la manière dont un patient a été diagnostiqué et la raison derrière un plan de traitement spécifique).

**Comment fonctionnent les valeurs SHAP ?**

Les valeurs SHAP interprètent l'impact d'avoir une certaine valeur pour une caractéristique donnée par rapport à la prédiction que nous ferions si cette caractéristique prenait une valeur de référence. Les valeurs SHAP le font de manière à garantir une belle propriété. Plus précisément, vous décomposez une prédiction avec l'équation suivante :

somme(des valeurs SHAP de toutes les caractéristiques) = prédiction_pour_réadmission - prédiction_pour_valeurs_de_base

C'est-à-dire que les valeurs SHAP de toutes les caractéristiques se cumulent pour expliquer pourquoi ma prédiction était différente de la valeur de base. Cela nous aide à expliquer pourquoi le modèle a identifié un patient pour une réadmission ou non.

--------------------------------------------------------------------------

In [ ]:
sample_patient = X_val.iloc[0].astype(float)

In [ ]:
# TODO: compléter la méthode main_factors() dans hw4.py.
main_factors(firstModel,sample_patient)

--------------------------------------------------

**Comment interprétez-vous cela ?**

Les valeurs des caractéristiques qui augmentent les prédictions sont en rouge, et leur taille visuelle montre l'ampleur de l'effet de la caractéristique. Les valeurs des caractéristiques qui diminuent la prédiction sont en bleu.

Si vous soustrayez la longueur des barres bleues de la longueur des barres rouges, cela équivaut à la distance de la valeur de base à la sortie.

Il y a une certaine complexité dans la technique, pour s'assurer que la valeur de base plus la somme des effets individuels s'ajoutent à la prédiction (ce qui n'est pas aussi simple que cela en a l'air). Nous n'entrerons pas dans les détails ici, car ce n'est pas essentiel pour l'utilisation de la technique. Cet article de blog propose une explication théorique plus longue : https://towardsdatascience.com/one-feature-attribution-method-to-supposedly-rule-them-all-shapley-values-f3e04534983d

----------------------------------------

**Question (Bonus)**

Comment interpréteriez-vous la visualisation des valeurs SHAP que nous avons obtenue ? Quels facteurs de risque augmentent le taux de réadmission et lesquels le diminuent réellement ?

*Votre réponse ici*

# Partie 2 : Sélection de Caractéristiques et Ingénierie des Caractéristiques

Dans cette deuxième partie de l'exercice, nous explorerons les sujets de la sélection de caractéristiques et de l'ingénierie des caractéristiques. Nous travaillerons avec un ensemble de données de prévision des tarifs de taxi de la ville de New York. Cet ensemble de données est plus gérable que notre ensemble de données précédent et nous permettra d'effectuer une sélection de caractéristiques et une ingénierie des caractéristiques (sans avoir besoin de posséder une connaissance avancée du domaine). Cependant, nous profiterons des fonctions implémentées dans la première partie de l'exercice pour accélérer le processus d'analyse, aider à la sélection des caractéristiques les plus pertinentes et identifier l'impact de la création d'une nouvelle caractéristique.

## 1. Gestion des Valeurs Aberrantes et Sélection de Caractéristiques

### 1.1 Chargement de l'Ensemble de Données

In [ ]:
data = load_data("data/ny_taxi.csv")

In [ ]:
data.head()

In [ ]:
data.describe()

In [ ]:
# Vérifiez s'il existe des valeurs manquantes
data.isna().any(axis=1).sum() 

### 1.2 Gestion des Valeurs Aberrantes et Sélection de Caractéristiques

Nous allons effectuer une détection et une suppression des valeurs aberrantes. Pour ce faire, vous devrez compléter la fonction `remove_outliers_iqr()` dans le fichier `hw4.py`. Cette fonction appliquera la méthode de la plage interquartile (IQR) pour la détection des valeurs aberrantes. La méthode IQR définit les valeurs aberrantes comme des points de données qui tombent en dessous de Q1 - 1,5 * IQR ou au-dessus de Q3 + 1,5 * IQR, où Q1 et Q3 sont les 25e et 75e percentiles, respectivement.

Ici, nous déciderons de travailler avec les caractéristiques numériques associées aux positions de prise en charge et de dépose et au nombre de passagers, nous ignorerons donc les colonnes `key` et `pickup_datetime`.

In [ ]:
# Les caractéristiques que nous examinerons pour les valeurs aberrantes :
base_features = ['pickup_longitude',
                 'pickup_latitude',
                 'dropoff_longitude',
                 'dropoff_latitude',
                 'passenger_count']

Nous utiliserons notre fonction `remove_outlier_iqr()`.

In [ ]:
# TODO : compléter la méthode remove_outliers_iqr() dans hw4.py.
clean_data = remove_outliers_iqr(data,base_features,"fare_amount")

**Question 2**

2-. Quels sont les avantages et les inconvénients de la méthode IQR que nous avons mise en œuvre ?

*Votre réponse ici*

In [ ]:
clean_data.head()

In [ ]:
clean_data.describe()

Maintenant, nous allons diviser notre ensemble de données en ensembles d'entraînement et de validation. Nous ne travaillerons qu'avec les colonnes indiquées dans la variable `selected_columns`.

In [ ]:
selected_columns = ['fare_amount','pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude','passenger_count']

X_train_2, X_val_2, y_train_2, y_val_2 = split_data(clean_data[selected_columns], "fare_amount")

In [ ]:
X_train_2.head()

In [ ]:
y_train_2.head()

Next, we will train our random forest regressor model.

In [ ]:
secondModel = RandomForestRegressor(n_estimators=50, random_state=1).fit(X_train_2, y_train_2)

**Question 3**

3-. (Sans effectuer aucune analyse) Quelles caractéristiques semblent potentiellement utiles pour prédire les tarifs de taxi ?

*Votre réponse ici*

Utilisons notre fonction `compute_permutation_importance()` pour découvrir quelles caractéristiques sont utilisées par le modèle pour effectuer la prédiction.

In [ ]:
perm_2 = calculate_permutation_importance(secondModel,X_val_2, y_val_2)
eli5.show_weights(perm_2, feature_names = X_val_2.columns.tolist())

Avant de voir ces résultats, nous aurions pu nous attendre à ce que chacune des 4 caractéristiques directionnelles soit également importante.

Cependant, en moyenne, les caractéristiques de latitude ont plus d'importance que les caractéristiques de longitude.

Nous remarquons également que nous pouvons cesser de travailler avec la caractéristique `passenger_count`, car il semble que ce soit la moins pertinente parmi nos caractéristiques.

**Question 4**

4-. Pouvez-vous émettre des hypothèses à ce sujet ?

*Votre réponse ici*

### 1.3 Ingénierie des Caractéristiques

Sans une connaissance détaillée de la ville de New York, il est difficile d'exclure la plupart des hypothèses sur la raison pour laquelle les caractéristiques de latitude ont plus d'importance que les caractéristiques de longitude.

Une bonne prochaine étape consiste à dissocier l'effet d'être dans certaines parties de la ville de l'effet de la distance totale parcourue.

Nous allons effectuer une ingénierie des caractéristiques, ce qui signifie que nous allons créer de nouvelles caractéristiques pour la distance longitudinale et latitudinale. Ensuite, nous construirons un nouveau modèle pour prédire le montant de la course, qui utilisera les caractéristiques existantes et les nouvelles caractéristiques que nous avons créées.

Complétez la fonction `add_absolute_coordinate_changes()` dans `hw4.py`. Elle devrait prendre notre cadre de données `clean_data` et renvoyer un nouveau cadre de données avec deux nouvelles colonnes appelées `abs_lat_change` et `abs_lon_change`.

In [ ]:
# TODO : compléter la méthode add_absolute_coordinate_changes() dans hw4.py.
new_c_data = add_absolute_coordinate_changes(clean_data)

In [ ]:
new_c_data.head()

In [ ]:
features_2  = ['fare_amount','pickup_longitude',
               'pickup_latitude',
               'dropoff_longitude',
               'dropoff_latitude',
               'abs_lat_change',
               'abs_lon_change']
X_train_3, X_val_3, y_train_3, y_val_3 = split_data(new_c_data[features_2], "fare_amount")

In [ ]:
X_train_3.head()

Nous allons entraîner un deuxième modèle avec les nouvelles caractéristiques.

In [ ]:
thirdModel = RandomForestRegressor(n_estimators=30, random_state=1).fit(X_train_3, y_train_3)

In [ ]:
perm_3 = calculate_permutation_importance(thirdModel, X_val_3, y_val_3)
eli5.show_weights(perm_3, feature_names = X_val_3.columns.tolist())

**Question 5**

5-. Comment interpréteriez-vous ces scores d'importance ? Il semble que la distance parcourue soit beaucoup plus importante que les effets de localisation.

*Votre réponse ici*

**Question 6**

6-. Vous avez vu que l’importance des caractéristiques pour la distance latitudinale est plus grande que l’importance de la distance longitudinale. De là, pouvons-nous conclure si voyager sur une distance latitudinale fixe tend à être plus cher que de voyager sur la même distance longitudinale ?

*Votre réponse ici*